# TNFR Molecule Atlas — Diatomic Demo

This notebook composes simple diatomic molecules by merging element-like TNFR graphs and adding coupling edges (telemetry-only).
It computes the Structural Field Tetrad (Φ_s, |∇φ|, K_φ, ξ_C), performs a sequential ΔΦ_s check (U6 telemetry), derives a descriptive signature, and saves HTML/CSV/JSONL outputs.

All content is in English and designed for canonical, reproducible runs.

In [ ]:
# Imports and configuration (read-only telemetry)
from __future__ import annotations

import os, json
from typing import Any, Dict, List, Tuple

import numpy as np
import pandas as pd

from tnfr.examples_utils import build_diatomic_molecule_graph, apply_synthetic_activation_sequence
from tnfr.physics.fields import (
    compute_structural_potential,
    compute_phase_gradient,
    compute_phase_curvature,
    estimate_coherence_length,
 )
from tnfr.operators.grammar import (
    warn_phase_gradient_telemetry,
    warn_phase_curvature_telemetry,
    warn_coherence_length_telemetry,
    validate_structural_potential_confinement,
 )
from tnfr.telemetry.constants import (
    PHASE_GRADIENT_THRESHOLD,
    PHASE_CURVATURE_ABS_THRESHOLD,
    STRUCTURAL_POTENTIAL_DELTA_THRESHOLD,
)

OUTPUT_DIR = os.path.join('examples', 'output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

def _symbol(Z: int) -> str:
    table = {1:'H',2:'He',3:'Li',4:'Be',5:'B',6:'C',7:'N',8:'O',9:'F',10:'Ne'}
    return table.get(Z, str(Z))

PAIRS: List[Tuple[int, int]] = [(1,1), (9,9), (3,9)]  # H2, F2, LiF

In [ ]:
# Signature derivation (telemetry-only; descriptive, not prescriptive)
def derive_signature(u6_ok: bool, mean_grad: float, mean_kphi: float, xi_c_val: float, mean_dist: float, loc_frac: float) -> str:
    if not u6_ok:
        return 'runaway'
    if mean_dist > 0.0 and xi_c_val > 3.0 * mean_dist:
        return 'critical'
    if (
        mean_grad < 0.8 * PHASE_GRADIENT_THRESHOLD
        and mean_kphi < 0.8 * PHASE_CURVATURE_ABS_THRESHOLD
        and (mean_dist == 0.0 or xi_c_val < mean_dist)
        and loc_frac >= 0.8
    ):
        return 'localized-confined'
    if (
        mean_grad >= PHASE_GRADIENT_THRESHOLD
        or mean_kphi >= PHASE_CURVATURE_ABS_THRESHOLD
        or loc_frac < 0.6
    ):
        return 'stressed'
    return 'confined'

In [ ]:
# Analysis helper for a diatomic pair (read-only telemetry)
def analyze_molecule(Z1: int, Z2: int, *, seed: int = 123) -> Dict[str, Any]:
    G = build_diatomic_molecule_graph(Z1, Z2, seed=seed, bond_links=1)

    # Canonical Tetrad telemetry
    phi_before = compute_structural_potential(G)
    grad = compute_phase_gradient(G)
    kphi = compute_phase_curvature(G)
    xi_c = float(estimate_coherence_length(G))

    # Non-blocking U6 telemetry warnings
    _, stats_g, msg_g, _ = warn_phase_gradient_telemetry(G, threshold=PHASE_GRADIENT_THRESHOLD)
    _, stats_k, msg_k, _ = warn_phase_curvature_telemetry(G, abs_threshold=PHASE_CURVATURE_ABS_THRESHOLD, multiscale_check=True, alpha_hint=2.76)
    _, stats_x, msg_x = warn_coherence_length_telemetry(G)

    # Sequential ΔΦ_s check (telemetry)
    apply_synthetic_activation_sequence(G, alpha=0.25, dnfr_factor=0.9)
    phi_after = compute_structural_potential(G)
    ok, drift, msg_u6 = validate_structural_potential_confinement(
        G, phi_before, phi_after, threshold=STRUCTURAL_POTENTIAL_DELTA_THRESHOLD, strict=False
    )

    # Locality fraction under thresholds
    total = max(1, len(G.nodes()))
    local_count = sum(
        1 for n in G.nodes()
        if abs(float(grad.get(n, 0.0))) < PHASE_GRADIENT_THRESHOLD
        and abs(float(kphi.get(n, 0.0))) < PHASE_CURVATURE_ABS_THRESHOLD
    )

    row = {
        'Z1': Z1,
        'Z2': Z2,
        'formula': f
,
        'xi_c': xi_c,
        'mean_grad': float(stats_g.get('mean_abs', 0.0)),
        'mean_kphi': float(stats_k.get('mean_abs', 0.0)),
        'mean_path_length': float(stats_x.get('mean_path_length', 0.0)),
        'u6_ok': bool(ok),
        'u6_drift': float(drift),
        'u6_msg': msg_u6,
        'local_frac': float(local_count) / float(total),
        'telemetry_msgs': [msg_g, msg_k, msg_x],
    }
    row['signature'] = derive_signature(
        row['u6_ok'], row['mean_grad'], row['mean_kphi'], row['xi_c'], row['mean_path_length'], row['local_frac']
    )
    return row

In [ ]:
# Run the atlas for selected diatomic pairs
rows: List[Dict[str, Any]] = [analyze_molecule(a, b) for (a, b) in PAIRS]
df = pd.DataFrame(rows)
df[['formula','signature','xi_c','mean_grad','mean_kphi','mean_path_length','local_frac','u6_ok','u6_drift']].sort_values('formula')

In [ ]:
# Save CSV and JSONL outputs for parity with the script
csv_path = os.path.join(OUTPUT_DIR, 'molecule_atlas.csv')
jsonl_path = os.path.join(OUTPUT_DIR, 'molecule_atlas.jsonl')
df[['formula','signature','xi_c','mean_grad','mean_kphi','mean_path_length','local_frac','u6_ok','u6_drift']].to_csv(csv_path, index=False)
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")
csv_path, jsonl_path

In [ ]:
# Summary by signature (telemetry-only classification) and save to disk
summary = (df.groupby('signature').size().reset_index(name='count').sort_values(['count','signature'], ascending=[False, True]))
summary_csv = os.path.join(OUTPUT_DIR, 'molecule_atlas_by_signature.csv')
summary_json = os.path.join(OUTPUT_DIR, 'molecule_atlas_summary.json')
summary.to_csv(summary_csv, index=False)
with open(summary_json, 'w', encoding='utf-8') as f:
    json.dump({ 'by_signature': summary.to_dict(orient='records') }, f, ensure_ascii=False, indent=2)
summary